In [1]:
using PhotonPropagation
using PhotonSurrogateModel
using NeutrinoTelescopeBase
using NeutrinoSurrogateModelData
using PhysicsTools
using StaticArrays
using LinearAlgebra
using Random
using Flux
using DataFrames
using CairoMakie
using BSON
using LogExpFunctions
using Distributions

In [2]:
model = BSON.load("/home/wecapstor3/capn/capn100h/simple_mlp_amp_model.bson")[:model]

SimpleMLPAmpModel(Chain(Dense(6 => 512, relu), Dropout(0.1, active=false), Dense(512 => 512, relu), Dropout(0.1, active=false), Dense(512 => 512, relu), Dropout(0.1, active=false), Dense(512 => 16)))

In [3]:
feat_buffer = zeros(Float32, 6, 100)

6×100 Matrix{Float32}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [4]:
get_pmt_count(POM(SA[0., 0., 0.], 1);)

MethodError: MethodError: no method matching get_pmt_count(::POM{Float64, POMAcceptance{Interpolations.FilledExtrapolation{Float64, 1, Interpolations.GriddedInterpolation{Float64, 1, Vector{Float64}, Interpolations.Gridded{Interpolations.Linear{Interpolations.Throw{Interpolations.OnGrid}}}, Tuple{Vector{Float64}}}, Interpolations.Gridded{Interpolations.Linear{Interpolations.Throw{Interpolations.OnGrid}}}, Float64}}, POMQuantumEff{Interpolations.FilledExtrapolation{Float64, 1, Interpolations.GriddedInterpolation{Float64, 1, Vector{Float64}, Interpolations.Gridded{Interpolations.Linear{Interpolations.Throw{Interpolations.OnGrid}}}, Tuple{Vector{Float64}}}, Interpolations.Gridded{Interpolations.Linear{Interpolations.Throw{Interpolations.OnGrid}}}, Float64}}})
The function `get_pmt_count` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  get_pmt_count(!Matched::Type{SphericalMultiPMTTarget{N, L}}) where {N, L}
   @ NeutrinoTelescopeBase /ho

In [5]:
methods(get_pmt_count)

# 3 methods for generic function "get_pmt_count" from NeutrinoTelescopeBase:
 [1] get_pmt_count(::Type{SphericalMultiPMTTarget{N, L}}) where {N, L}
     @ /home/saturn/capn/capn100h/julia_depot/packages/NeutrinoTelescopeBase/Eza5j/src/photon_targets.jl:199
 [2] get_pmt_count(::SphericalMultiPMTTarget{N, L}) where {N, L}
     @ /home/saturn/capn/capn100h/julia_depot/packages/NeutrinoTelescopeBase/Eza5j/src/photon_targets.jl:198
 [3] get_pmt_count(::HomogeneousTarget)
     @ /home/saturn/capn/capn100h/julia_depot/packages/NeutrinoTelescopeBase/Eza5j/src/photon_targets.jl:197

In [5]:
dir_ct = 0.2
dir_phis = 0:0.01:2π
target = POM(SA[0., 0., 0.], 1);

amps = []

for dir_phi in dir_phis
    pdir = sph_to_cart(acos(dir_ct), dir_phi)
    particle = Particle(SA[0., 10, 0], pdir, 0., 4E3, 0., PEMinus)
    log_amps = get_log_amplitudes([particle], [target], model, feat_buffer=feat_buffer, device=cpu)
    push!(amps, exp.(log_amps[1]))
end
    

MethodError: MethodError: no method matching get_pmt_count(::Type{POM{Float64, POMAcceptance{Interpolations.FilledExtrapolation{Float64, 1, Interpolations.GriddedInterpolation{Float64, 1, Vector{Float64}, Interpolations.Gridded{Interpolations.Linear{Interpolations.Throw{Interpolations.OnGrid}}}, Tuple{Vector{Float64}}}, Interpolations.Gridded{Interpolations.Linear{Interpolations.Throw{Interpolations.OnGrid}}}, Float64}}, POMQuantumEff{Interpolations.FilledExtrapolation{Float64, 1, Interpolations.GriddedInterpolation{Float64, 1, Vector{Float64}, Interpolations.Gridded{Interpolations.Linear{Interpolations.Throw{Interpolations.OnGrid}}}, Tuple{Vector{Float64}}}, Interpolations.Gridded{Interpolations.Linear{Interpolations.Throw{Interpolations.OnGrid}}}, Float64}}}})
The function `get_pmt_count` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  get_pmt_count(!Matched::Type{SphericalMultiPMTTarget{N, L}}) where {N, L}
   @ NeutrinoTelescopeBa

In [6]:
fig = Figure(size=(1000, 1000))
axes = []
for pmt_ix in 1:16
    row, col = divrem(pmt_ix-1, 4)
    ax = Axis(fig[row, col], xlabel="phi", ylabel="amp")
    push!(axes, ax)
end


for pmt_ix in 1:16
    this_amps = [a[pmt_ix] for a in amps]
    lines!(axes[pmt_ix], dir_phis, this_amps)
end
fig




ArgumentError: ArgumentError: reducing over an empty collection is not allowed; consider supplying `init` to the reducer

In [28]:
size(log_amps[1])

(16, 1, 1)